In [1]:
import bladerf
from bladerf import _bladerf
import matplotlib.pyplot as plt

In [4]:
d = bladerf.BladeRF()


ch_rx_0 = d.Channel(bladerf.CHANNEL_RX(0))
ch_rx_1 = d.Channel(bladerf.CHANNEL_RX(1))
ch_tx_0 = d.Channel(bladerf.CHANNEL_TX(0))
ch_tx_1 = d.Channel(bladerf.CHANNEL_TX(1))



d.get_gain_modes(ch_rx_0)



d.close()



NoDevError: No devices available

In [ ]:
sig = bladerf.TRIGGER_SIGNAL.J51_1.value


masterChannel = bladerf.CHANNEL_TX(0)

masterRole = bladerf.TRIGGER_ROLE.Master.value

masterTrigger = d.master_trigger_init(masterChannel, sig)



slaveChannel = bladerf.CHANNEL_RX(0)

slaveRole = bladerf.TRIGGER_ROLE.Slave.value

slaveTrigger = d.trigger_init(slaveChannel, slaveRole, sig)



In [ ]:
print("Arming Both Triggers")
d.trigger_arm(masterTrigger, True)
d.trigger_arm(slaveTrigger, True)


d.trigger_state(masterTrigger)
d.trigger_state(slaveTrigger)

d.sync_config(layout = _bladerf.ChannelLayout.RX_X1,
                       fmt            = _bladerf.Format.SC16_Q11,
                       num_buffers    = 16,
                       buffer_size    = 8192,
                       num_transfers  = 8,
                       stream_timeout = 10000)

s = d.Channel(bladerf.CHANNEL_RX(0))
s.enable = True


# d.sync_rx()
bytes_per_sample = 4
buf = bytearray(1024*bytes_per_sample)

num_samples = 1024


In [ ]:
d.trigger_fire(masterTrigger)

d.sync_rx(buf, num_samples=num_samples)

# Disable module
print( "RX: Stop" )
s.enable = False

print( "RX: Done" )

In [ ]:
import numpy as np

In [ ]:
dtype_data = np.dtype([('re', np.int16), ('im', np.int16)])

proper_data = np.asanyarray(buf, dtype=dtype_data)

math_data = proper_data.view(np.int16).astype(np.float32).view(np.complex64)

default_data = np.asanyarray(buf, dtype=complex)


In [ ]:
print("Default Format: {}".format(default_data[:5]))
print("")
print("Proper Format: {}".format(proper_data[:5]))
print("")
print("Math Format: {}".format(math_data[:5]))

In [ ]:

def plotPSD(data, fftWindow, Fs, Fc):
    assert fftWindow in ['rectangular', 'bartlett', 'blackman',
                         'hamming', 'hanning']

    N = len(data)

    # Generate the selected window
    if fftWindow == "rectangular":
        window = np.ones(N)
    elif fftWindow == "bartlett":
        window = np.bartlett(N)
    elif fftWindow == "blackman":
        window = np.blackman(N)
    elif fftWindow == "hamming":
        window = np.hamming(N)
    elif fftWindow == "hanning":
        window = np.hanning(N)

    dft = np.fft.fft(data * window)

    # if Fs == None:
    #     # If the sample rate is known then plot PSD as
    #     # Power/Freq in (dB/Hz)
    plt.psd(data * window, NFFT=N, Fs=Fs, Fc=Fc)
    #     print("first")
    # else:
    #     # If sample rate is not known then plot PSD as
    #     # Power/Freq as (dB/rad/sample)
    #     plt.psd(data * window, NFFT=N, Fs=Fs)
    #     print("second")
    plt.show()

In [ ]:
plotPSD(math_data, fftWindow='rectangular', Fs=20e6, Fc=3e9)